In [15]:
import numpy as np
import pandas as pd

Products URL is considered the "gold-standard". We need to look through the log file to 1) find corrupted URL's, and 2) write and algorithm to correct them. 

In [10]:
# Products URL should be considered "gold standard"

products_df = pd.read_csv('../data/products.csv')
log_df = pd.read_csv('../data/log2.csv',
                     names=['sentiment', 'publication_URL', 'product_URL',
                     'clickORnot', 'gender', 'age_group'])
product_categories = pd.read_csv('../data/product_categories.csv')

### Question 1
*Some of the Product_URLs in the log file might have been corrupted. Write a Python (or PySpark) procedure to determine which Product_URLs are corrupted. Let us assume that if a Product_url in the log file doesn’t occur in the products table, it is regarded as corrupted. Using this procedure identify and list the corrupted URLs. (10)*

The code below uses the simple list matching function `np.in1d()` to quickly identify which URLs in the log file are not present in hte products file.

In [29]:
# Pull out URLs and strip leading/lagging whitespace
product_urls = products_df['product_URL'].str.strip().values
log_urls = log_df['product_URL'].str.strip().values

# Check which elements of log_urls are NOT an exact match in product_urls
url_mask = ~np.in1d(log_urls, product_urls)
log_df[url_mask]

,sentiment,publication_URL,product_URL,clickORnot,gender,age_group
83,negative,https://www.cbsnews.com/,https://haier.com/refrigermtors,0,female,young
109,neutral,https://mashable.com/,https://sony.comftelevisions,1,female,juvenile
123,negative,https://www.thedailybeast.com/,https://lg.com/gashers,0,female,middle-age
171,neutral,https://www.cnn.com/,https://leks.com/jeans,0,female,senior
203,neutral,https://www.nytimes.com/,https://InstantPot.con/cookers,1,female,young
...,...,...,...,...,...,...
9729,neutral,https://www.usnews.com/,http://nejoK.co/blenders,0,male,senior
9773,negative,https://www.nbcnews.com/,https://maytag.cpm/washers,1,male,middle-age
9845,positive,https://www.foxnews.com/,https://guessmcom/perfumes,1,female,middle-age
9881,neutral,https://www.usnews.com/,https://samsuag.com/televisions,0,female,young


### Question 2
For each corrupted URL what will you do with it? Don’t assume that for each corrupted URL the correct approach is to delete that log entry. What if the URL contained ‘.cam’ instead of ‘.com’ but otherwise corresponded with a URL in the ‘products’ table? In that case the proper approach would be to correct the URL. In other cases, the URL might be so corrupted that the best approach would be to delete that log entry (the entire row). Describe your approach to dealing with corrupted URLs. That is, describe your approach to determining that a URL is too corrupted to be rescued. It must describe a) a procedure for determining the degree to which the URL is corrupted, b) a threshold for determining in terms of this degree of corruption whether it can be corrected, and c) for those which can be corrected, identifying its corrected form. For extra credit implement this in a Python (or PySpark) program. (25 + 20 points for extra-credit)

In [7]:
# Taken from: https://stackoverflow.com/questions/3809401/what-is-a-good-regular-expression-to-match-a-url


NameError: name 'log' is not defined

In [8]:
import numpy as np
from scipy.stats import binom
import matplotlib.pyplot as plt